In [ ]:
%matplotlib notebook

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
from disba import PhaseDispersion, GroupDispersion
from disba import depthplot
from Model import Model

In [ ]:
periods = np.logspace(0.0, 3., 100)
periods = periods[periods < 700]

#f = np.linspace(0.04, 0.18, 30)
#periods = 1/f[::-1]

In [ ]:
prem = Model("/usr/local/TauP-2.1.1/StdModels/prem.nd")
prem.insert_prem_ocean()

In [ ]:
f , (ax1, ax2) = plt.subplots(1,2, figsize = (5,4))

f = depthplot(*prem.thick_get('vs'),
              plot_args = { 'label' : 'Vs', 'ls' :'-', 'color' : 'red' },
              ax = ax1)

f = depthplot(*prem.thick_get('vp'),
              plot_args = { 'label' : 'Vp', 'ls' :'-', 'color' : 'red' },
              ax = ax2)


for d in np.arange(70,200,20):
    _m = prem.mod({
        'l' : (None, d, False),
        'u' : (d, None, False)    
    })
    f = depthplot(*_m.thick_get('vs'),
                  plot_args = { 'label' : 'Vs', 'ls':'--', 'color' : 'gray', 'linewidth' : 1},
                  ax = ax1)
    f = depthplot(*_m.thick_get('vp'),
                  plot_args = { 'label' : 'Vp','ls':'--', 'color' : 'gray', 'linewidth' : 1 },
                  ax = ax2)

ax1.set_title('S-Wave')
ax1.set_ylim([410,0])
ax1.set_xlim([4.2,4.8])
ax1.set_ylabel('Depth [km]')
ax1.set_xlabel('Velocity [km/s]')

ax2.set_title('P-Wave')
ax2.set_ylim([410,0])
ax2.set_xlim([7.5,9])
ax2.set_ylabel('Depth [km]')
ax2.set_xlabel('Velocity [km/s]')
plt.tight_layout()

In [ ]:
_,_,f_per,_,f_C,f_U,_,_,_ = np.loadtxt("prem-R.asc", skiprows = 1, unpack = True)
_,_,fo_per,_,fo_C,fo_U,_,_,_ = np.loadtxt("prem-ocean-R.asc", skiprows = 1, unpack = True)

#_,_,f_per,_,f_C,f_U,_,_ = np.loadtxt("prem-L.asc", skiprows = 1, unpack = True)
#_,_,fo_per,_,fo_C,fo_U,_,_ = np.loadtxt("prem-ocean-L.asc", skiprows = 1, unpack = True)

In [ ]:
# MM = 2850
MM = None

In [ ]:
f , (ax1, ax2) = plt.subplots(1,2, figsize = (8,4))

ax1.axhspan(0.0, MM if MM is not None else 6371, alpha = 0.2)

depthplot(*prem.thick_get('vs'),
              plot_args = { 'label' : 'Vs' } ,
              ax = ax1)

for dep in np.arange(30,230,30):
    _m = prem.mod({
        'l' : (None, dep, True),
        'u' : (dep, None, True)
        })
    
    d,vp,vs,rho = _m.thick_get(['vp', 'vs', 'rho'], max_depth = MM)
    GpremF   = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
    Rg_premF = GpremF(periods, 0, 'rayleigh')

    ax2.plot(Rg_premF.period, Rg_premF.velocity,
             '-', label = 'PREM @ %d' % dep)

    depthplot(*_m.thick_get('vs'),
                  plot_args = { 'label' : 'Vs', 'linestyle' : '--', 'linewidth' : 1 },
                  ax = ax1)

#ax2.plot(f_per, f_C, label = 'Herman C')
#ax2.plot(f_per, f_U, '--', label = 'Herman U', lw = 2, color = 'k')

#ax2.plot(fo_per, fo_C, label = 'Herman Ocean C')
ax2.plot(fo_per, fo_U, '--', label = 'Herman Ocean U', lw = 2, color = 'k')

ax1.set_xlabel('S-wave velocity [km/s]')
ax1.set_ylabel('Depth [km]')
ax1.set_ylim((410, 0 ))
ax1.set_xlim((3, 5.1 ))

ax2.set_xscale("log", base=10)
#ax2.set_yscale("log", base=10)

ax2.set_xlabel('Period [s]')
#ax2.set_xlim(1,1000)
ax2.set_ylabel('Velocity [km/s]')

ax2.legend()

plt.tight_layout()

In [ ]:
f , (ax1, ax2) = plt.subplots(1,2, figsize = (8,4))

ax1.axhspan(0.0, MM if MM is not None else 6371, alpha = 0.2)

_m = prem.mod({
    'l' : (None, 120, True),
    'u' : (120, None, True)
    })

depthplot(*_m.thick_get('vs'),
              plot_args = { 'label' : 'Vs' } ,
              ax = ax1)

d,vp,vs,rho = _m.thick_get(['vp', 'vs', 'rho'], max_depth = MM)
GpremF   = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
ref = GpremF(periods, 0, 'rayleigh')

for dep in np.arange(30,230,30):
    _m = prem.mod({
        'l' : (None, dep, True),
        'u' : (dep, None, True)
        })
    
    d,vp,vs,rho = _m.thick_get(['vp', 'vs', 'rho'], max_depth = MM)
    GpremF   = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
    Rg_premF = GpremF(periods, 0, 'rayleigh')

    ax2.plot(Rg_premF.period, 100 * (ref.velocity - Rg_premF.velocity) / ref.velocity,
             '-', label = 'PREM @ %d' % dep)

    depthplot(*_m.thick_get('vs'),
                  plot_args = { 'label' : 'Vs', 'linestyle' : '--', 'linewidth' : 1 },
                  ax = ax1)

#ax2.plot(f_per, f_C, label = 'Herman C')
#ax2.plot(f_per, f_U, label = 'Herman U')

#ax2.plot(fo_per, fo_C, label = 'Herman Ocean C')
#ax2.plot(fo_per, fo_U, label = 'Herman Ocean U')

ax1.set_xlabel('S-wave velocity [km/s]')
ax1.set_ylabel('Depth [km]')
ax1.set_ylim((410, 0 ))
ax1.set_xlim((3, 5.1 ))

ax2.set_xscale("log", base=10)
#ax2.set_yscale("log", base=10)

ax2.set_xlabel('Period [s]')
#ax2.set_xlim(1,1000)
ax2.set_ylabel(r'$\delta_{Vs}$ %')

ax2.legend()

plt.tight_layout()

In [ ]:
 m = Model("simple.tvel")
_m = Model("simple2.tvel")

In [ ]:
f , (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (8,4))

depthplot(*m.thick_get('vs'),
              plot_args = { 'label' : 'Vs @ 120 km'} ,
              ax = ax1)

d,vp,vs,rho = m.thick_get(['vp', 'vs', 'rho'])
GpremF    = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
Rg_premF1 = GpremF(periods, 0, 'rayleigh')
ax2.plot(Rg_premF1.period, Rg_premF1.velocity, '-', color = 'k')


depthplot(*_m.thick_get('vs'),
              plot_args = { 'label' : 'Vs @ 60 km', 'linestyle' : '--', 'linewidth' : 1 },
              ax = ax1)
d,vp,vs,rho = _m.thick_get(['vp', 'vs', 'rho'])
GpremF   = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
Rg_premF2 = GpremF(periods, 0, 'rayleigh')

ax2.plot(Rg_premF2.period, Rg_premF2.velocity, '--', color = 'k', linewidth = 1 )
ax3.plot(Rg_premF2.period, 100 * (Rg_premF2.velocity - Rg_premF1.velocity)/ Rg_premF1.velocity)

ax1.set_xlabel('S-wave velocity [km/s]')
ax1.set_ylabel('Depth [km]')
ax1.legend()
ax1.set_ylim((240, 0 ))
ax1.set_xlim((3, 5.1 ))

ax2.set_xscale("log", base=10)
#ax2.set_yscale("log", base=10)

ax2.set_xlabel('Period [s]')
#ax2.set_xlim(1,1000)
ax2.set_ylabel('Velocity [km/s]')

ax3.set_xscale("log", base=10)
ax3.set_xlabel('Period [s]')
ax3.set_ylabel(r'$\delta_{Vs}$ %')

plt.tight_layout()